In [1]:
# My own Convolutional Neural Network

# Tensorflow
import tensorflow as tf
import numpy as np

# Importing MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)


# Parameters
learning_rate = 0.000001
epochs = 1000
batch_size = 128

valid_data_size = 256

# Network Parameters
n_classes = 10
dropout = 0.5



# -------------------- def Xavier_initializer -------------------- #
def Xavier_init(name, shape_in):
    return tf.get_variable(name, shape=shape_in, initializer=tf.contrib.layers.xavier_initializer())
# ---------------------------------------------------------------- #


# Store Weights & Bias
weights = {
    'wc1': Xavier_init('wc1',[5, 5, 1, 32]),
    'wc2': Xavier_init('wc2',[5, 5, 32, 64]),
    'w1' : Xavier_init('w1',[7*7*64, 1024]), # 7*7*64
    'w2' : Xavier_init('w2',[1024, 1024]),
    'wout': Xavier_init('wout',[1024, n_classes])
}

biases = {
    'bc1': Xavier_init('bc1',[32]),
    'bc2': Xavier_init('bc2',[64]),
    'b1' : Xavier_init('b1',[1024]),
    'b2' : Xavier_init('b2',[1024]),
    'bout': Xavier_init('bout',[n_classes])
}

# ------------------------- defining functions ----------------------- #
# Convolution layer
def conv2d(x, W, b, strides=1):                                         # stride = 1
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding = 'SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

# Pooling
def maxpool2d(x, k=2):                                                  # patch size = 2
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')

# Convolution + Pooling + hidden layer
def conv_net(x,weights, biases, dropout):
    # Convolution Layer #1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    # Convolution Layer #2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    
    # Hidden Layer #1
    hl1 = tf.reshape(conv2, [-1, weights['w1'].get_shape().as_list()[0]])
    hl1 = tf.add(tf.matmul(hl1, weights['w1']), biases['b1'])
    hl1 = tf.nn.relu(hl1)
    hl1 = tf.nn.dropout(hl1, dropout)
    
  
    # Hidden Layer #2
    # hl2 = tf.reshape(hl1, [-1, weights['w2'].get_shape().as_list()[0]])
    hl2 = tf.add(tf.matmul(hl1, weights['w2']), biases['b2'])
    hl2 = tf.nn.relu(hl2)
    hl2 = tf.nn.dropout(hl2, dropout)
    
    # Output Layer
    out = tf.add(tf.matmul(hl2, weights['wout']), biases['bout'])





    return out, conv1, conv2, hl1, hl2
 
# --------------------------------------------------------------------------- #


# input & output placeholder
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob_dropout = tf.placeholder(tf.float32)


# Model : output
logits, conv1, conv2, fc1, fc2  = conv_net(x, weights, biases, keep_prob_dropout)

# Define loss & Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1)) # 1???? test later
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# initializer
init = tf.global_variables_initializer()

# Saver
save_file = './SaveData/test1.ckpt'
saver = tf.train.Saver()

# Training start
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            sess.run(optimizer, feed_dict={x:batch_x, y:batch_y, keep_prob_dropout: dropout}) # ...? drop out?
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob_dropout: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:valid_data_size],
                y: mnist.validation.labels[:valid_data_size],
                keep_prob_dropout: 1.})
            # print('Epoch = {:}, Batch = {:}, loss = {:.5f}, accuracy = {:.5f}'.format(epoch+1, batch+1, loss, valid_acc))
            
            # for debugging
            # print(sess.run(logits, feed_dict={x:batch_x, y:batch_y, keep_prob_dropout: dropout}))
            # conv11, conv21, fc11, fc21 = sess.run([conv1, conv2, fc1, fc2], feed_dict = {x:batch_x, y:batch_y, keep_prob_dropout: dropout})
            # print(sess.run(logits, feed_dict={x:batch_x, y:batch_y, keep_prob_dropout: dropout}))
            # print(np.any(np.isinf(conv11)),np.any(np.isnan(conv11)))
            # print(np.any(np.isinf(conv21)),np.any(np.isnan(conv21)))
            # print(np.any(np.isinf(fc11)),np.any(np.isnan(fc11)))
            # print(np.any(np.isinf(fc21)),np.any(np.isnan(fc21)))
        
        print('Epoch = {:}, Batch = {:}, loss = {:.5f}, accuracy = {:.5f}'.format(epoch+1, batch+1, loss, valid_acc))
        #saver.save(sess, save_file)
    print('fuck')
    


/home/smnd/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


KeyboardInterrupt: 

In [1]:
# ========================== My own Convolutional Neural Network =========================== #

# Tensorflow
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
# Importing MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)


# Parameters
learning_rate = 0.000001
epochs = 1000
batch_size = 128

valid_data_size = 256

# Network Parameters
n_classes = 10
dropout = 0.5

# -------------------- def Xavier_initializer -------------------- #
def Xavier_init(name, shape_in):
    return tf.get_variable(name, shape=shape_in, initializer=tf.contrib.layers.xavier_initializer())
# ---------------------------------------------------------------- #


# Store Weights & Bias
weights = {
    'wc1': Xavier_init('wc1',[5, 5, 1, 32]),
    'wc2': Xavier_init('wc2',[5, 5, 32, 64]),
    'w1' : Xavier_init('w1',[7*7*64, 1024]), # 7*7*64
    'w2' : Xavier_init('w2',[1024, 1024]),
    'wout': Xavier_init('wout',[1024, n_classes])
}

biases = {
    'bc1': Xavier_init('bc1',[32]),
    'bc2': Xavier_init('bc2',[64]),
    'b1' : Xavier_init('b1',[1024]),
    'b2' : Xavier_init('b2',[1024]),
    'bout': Xavier_init('bout',[n_classes])
}

# ------------------------- defining functions ----------------------- #
# Convolution layer
def conv2d(x, W, b, strides=1):                                         # stride = 1
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding = 'SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

# Pooling
def maxpool2d(x, k=2):                                                  # patch size = 2
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')

# Convolution + Pooling
def conv_net(x,weights, biases, dropout):
    # Convolution Layer #1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    # Convolution Layer #2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    
    # Hidden Layer #1
    hl1 = tf.reshape(conv2, [-1, weights['w1'].get_shape().as_list()[0]])
    hl1 = tf.add(tf.matmul(hl1, weights['w1']), biases['b1'])
    hl1 = tf.nn.relu(hl1)
    hl1 = tf.nn.dropout(hl1, dropout)
    
  
    # Hidden Layer #2
    # hl2 = tf.reshape(hl1, [-1, weights['w2'].get_shape().as_list()[0]])
    hl2 = tf.add(tf.matmul(hl1, weights['w2']), biases['b2'])
    hl2 = tf.nn.relu(hl2)
    hl2 = tf.nn.dropout(hl2, dropout)
    
    # Output Layer
    out = tf.add(tf.matmul(hl2, weights['wout']), biases['bout'])





    return out, conv1, conv2, hl1, hl2
 
# --------------------------------------------------------------------------- #


# input & output placeholder
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob_dropout = tf.placeholder(tf.float32)


# Model : output
logits, conv1, conv2, fc1, fc2  = conv_net(x, weights, biases, keep_prob_dropout)

# Define loss & Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1)) # 1???? test later
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# initializer
init = tf.global_variables_initializer()

# Saver
save_file = './SaveData/test1.ckpt'
saver = tf.train.Saver()

# Training start
with tf.Session() as sess:
    sess.run(init)
    # load
    saver.restore(sess, save_file)
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            sess.run(optimizer, feed_dict={x:batch_x, y:batch_y, keep_prob_dropout: dropout}) # ...? drop out?
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob_dropout: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:valid_data_size],
                y: mnist.validation.labels[:valid_data_size],
                keep_prob_dropout: 1.})
            
            
            # for debugging
            # print(sess.run(logits, feed_dict={x:batch_x, y:batch_y, keep_prob_dropout: dropout}))
            # conv11, conv21, fc11, fc21 = sess.run([conv1, conv2, fc1, fc2], feed_dict = {x:batch_x, y:batch_y, keep_prob_dropout: dropout})
            # print(sess.run(logits, feed_dict={x:batch_x, y:batch_y, keep_prob_dropout: dropout}))
            # print(np.any(np.isinf(conv11)),np.any(np.isnan(conv11)))
            # print(np.any(np.isinf(conv21)),np.any(np.isnan(conv21)))
            # print(np.any(np.isinf(fc11)),np.any(np.isnan(fc11)))
            # print(np.any(np.isinf(fc21)),np.any(np.isnan(fc21)))
        saver.save(sess, save_file)
        print('Epoch = {:}, Batch = {:}, loss = {:.5f}, accuracy = {:.5f}'.format(epoch+1, batch+1, loss, valid_acc))
    print('fuck')
    
            

/home/smnd/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from ./SaveData/test1.ckpt
Epoch = 1, Batch = 429, loss = 0.26446, accuracy = 0.92188
Epoch = 2, Batch = 429, loss = 0.24743, accuracy = 0.92188
Epoch = 3, Batch = 429, loss = 0.24046, accuracy = 0.92188
Epoch = 4, Batch = 429, loss = 0.28994, accuracy = 0.92188
Epoch = 5, Batch = 429, loss = 0.26842, accuracy = 0.92188
Epoch = 6, Batch = 429, loss = 0.24339, accuracy = 0.92578
Epoch = 7, Batch = 429, loss = 0.13569, accuracy = 0.92578
Epoch = 8, Batch = 429, loss = 0.19755, accuracy = 0.92578
Epoch = 9, Batch = 429, loss = 0.20549, accuracy = 0.92578
Epoch = 10, Batch = 429, loss = 0.28872, accuracy = 0.92188
Epoch = 11, Batch = 429, loss = 0.21335, accuracy = 0.92578
Epoch = 12, Batch = 429, loss = 0.15247, accuracy = 0.92969
Epoch = 13, Batch = 429, loss = 0.24080, accuracy = 0.92578
Epoch =

Epoch = 134, Batch = 429, loss = 0.06296, accuracy = 0.96875
Epoch = 135, Batch = 429, loss = 0.02481, accuracy = 0.96875
Epoch = 136, Batch = 429, loss = 0.11471, accuracy = 0.96875
Epoch = 137, Batch = 429, loss = 0.08539, accuracy = 0.96875
Epoch = 138, Batch = 429, loss = 0.08459, accuracy = 0.96875
Epoch = 139, Batch = 429, loss = 0.05398, accuracy = 0.96875
Epoch = 140, Batch = 429, loss = 0.07785, accuracy = 0.96875
Epoch = 141, Batch = 429, loss = 0.04470, accuracy = 0.97266
Epoch = 142, Batch = 429, loss = 0.12198, accuracy = 0.97266
Epoch = 143, Batch = 429, loss = 0.08166, accuracy = 0.97266
Epoch = 144, Batch = 429, loss = 0.04841, accuracy = 0.96875
Epoch = 145, Batch = 429, loss = 0.03980, accuracy = 0.97266
Epoch = 146, Batch = 429, loss = 0.04576, accuracy = 0.97266
Epoch = 147, Batch = 429, loss = 0.13336, accuracy = 0.97266
Epoch = 148, Batch = 429, loss = 0.14209, accuracy = 0.96875
Epoch = 149, Batch = 429, loss = 0.02905, accuracy = 0.97266
Epoch = 150, Batch = 429

Epoch = 269, Batch = 429, loss = 0.18364, accuracy = 0.97266
Epoch = 270, Batch = 429, loss = 0.02175, accuracy = 0.97266
Epoch = 271, Batch = 429, loss = 0.02138, accuracy = 0.97266
Epoch = 272, Batch = 429, loss = 0.06151, accuracy = 0.97266
Epoch = 273, Batch = 429, loss = 0.07962, accuracy = 0.97266
Epoch = 274, Batch = 429, loss = 0.05511, accuracy = 0.97266
Epoch = 275, Batch = 429, loss = 0.08406, accuracy = 0.97266
Epoch = 276, Batch = 429, loss = 0.03597, accuracy = 0.97266
Epoch = 277, Batch = 429, loss = 0.03026, accuracy = 0.97266
Epoch = 278, Batch = 429, loss = 0.02695, accuracy = 0.97266
Epoch = 279, Batch = 429, loss = 0.02085, accuracy = 0.97266
Epoch = 280, Batch = 429, loss = 0.03424, accuracy = 0.97266
Epoch = 281, Batch = 429, loss = 0.01733, accuracy = 0.97266
Epoch = 282, Batch = 429, loss = 0.05117, accuracy = 0.97266
Epoch = 283, Batch = 429, loss = 0.03238, accuracy = 0.97266
Epoch = 284, Batch = 429, loss = 0.05462, accuracy = 0.97266
Epoch = 285, Batch = 429

Epoch = 404, Batch = 429, loss = 0.02292, accuracy = 0.98047
Epoch = 405, Batch = 429, loss = 0.03537, accuracy = 0.98047
Epoch = 406, Batch = 429, loss = 0.03466, accuracy = 0.98047
Epoch = 407, Batch = 429, loss = 0.01668, accuracy = 0.98047
Epoch = 408, Batch = 429, loss = 0.05038, accuracy = 0.98047
Epoch = 409, Batch = 429, loss = 0.00861, accuracy = 0.98047
Epoch = 410, Batch = 429, loss = 0.01992, accuracy = 0.98047
Epoch = 411, Batch = 429, loss = 0.04833, accuracy = 0.98047
Epoch = 412, Batch = 429, loss = 0.00681, accuracy = 0.98047
Epoch = 413, Batch = 429, loss = 0.07548, accuracy = 0.98047
Epoch = 414, Batch = 429, loss = 0.01965, accuracy = 0.98047
Epoch = 415, Batch = 429, loss = 0.07042, accuracy = 0.98047
Epoch = 416, Batch = 429, loss = 0.00470, accuracy = 0.98047
Epoch = 417, Batch = 429, loss = 0.05652, accuracy = 0.98047
Epoch = 418, Batch = 429, loss = 0.04403, accuracy = 0.98047
Epoch = 419, Batch = 429, loss = 0.00298, accuracy = 0.98047
Epoch = 420, Batch = 429

Epoch = 539, Batch = 429, loss = 0.00185, accuracy = 0.98047
Epoch = 540, Batch = 429, loss = 0.01847, accuracy = 0.98047
Epoch = 541, Batch = 429, loss = 0.00833, accuracy = 0.98047
Epoch = 542, Batch = 429, loss = 0.00746, accuracy = 0.98047
Epoch = 543, Batch = 429, loss = 0.02045, accuracy = 0.98047
Epoch = 544, Batch = 429, loss = 0.00541, accuracy = 0.98047
Epoch = 545, Batch = 429, loss = 0.04548, accuracy = 0.98047
Epoch = 546, Batch = 429, loss = 0.01248, accuracy = 0.98047
Epoch = 547, Batch = 429, loss = 0.01269, accuracy = 0.98047
Epoch = 548, Batch = 429, loss = 0.01263, accuracy = 0.98047
Epoch = 549, Batch = 429, loss = 0.00685, accuracy = 0.98047
Epoch = 550, Batch = 429, loss = 0.02823, accuracy = 0.98047
Epoch = 551, Batch = 429, loss = 0.01487, accuracy = 0.98047
Epoch = 552, Batch = 429, loss = 0.02470, accuracy = 0.98047
Epoch = 553, Batch = 429, loss = 0.01142, accuracy = 0.98047
Epoch = 554, Batch = 429, loss = 0.00333, accuracy = 0.98047
Epoch = 555, Batch = 429

Epoch = 674, Batch = 429, loss = 0.00227, accuracy = 0.98047
Epoch = 675, Batch = 429, loss = 0.00431, accuracy = 0.98047
Epoch = 676, Batch = 429, loss = 0.00450, accuracy = 0.98047
Epoch = 677, Batch = 429, loss = 0.00108, accuracy = 0.98047
Epoch = 678, Batch = 429, loss = 0.01865, accuracy = 0.98047
Epoch = 679, Batch = 429, loss = 0.03083, accuracy = 0.98047
Epoch = 680, Batch = 429, loss = 0.00573, accuracy = 0.98047
Epoch = 681, Batch = 429, loss = 0.00268, accuracy = 0.98047
Epoch = 682, Batch = 429, loss = 0.03635, accuracy = 0.98047
Epoch = 683, Batch = 429, loss = 0.00343, accuracy = 0.98047
Epoch = 684, Batch = 429, loss = 0.01401, accuracy = 0.98047
Epoch = 685, Batch = 429, loss = 0.00792, accuracy = 0.98047
Epoch = 686, Batch = 429, loss = 0.00246, accuracy = 0.98047
Epoch = 687, Batch = 429, loss = 0.00126, accuracy = 0.98047
Epoch = 688, Batch = 429, loss = 0.00828, accuracy = 0.98047
Epoch = 689, Batch = 429, loss = 0.00628, accuracy = 0.98047
Epoch = 690, Batch = 429

Epoch = 809, Batch = 429, loss = 0.00155, accuracy = 0.98047
Epoch = 810, Batch = 429, loss = 0.00401, accuracy = 0.98047
Epoch = 811, Batch = 429, loss = 0.02247, accuracy = 0.98047
Epoch = 812, Batch = 429, loss = 0.02420, accuracy = 0.98047
Epoch = 813, Batch = 429, loss = 0.01183, accuracy = 0.98047
Epoch = 814, Batch = 429, loss = 0.00598, accuracy = 0.98047
Epoch = 815, Batch = 429, loss = 0.03817, accuracy = 0.98047
Epoch = 816, Batch = 429, loss = 0.00367, accuracy = 0.98047
Epoch = 817, Batch = 429, loss = 0.00217, accuracy = 0.98047
Epoch = 818, Batch = 429, loss = 0.00303, accuracy = 0.98047
Epoch = 819, Batch = 429, loss = 0.00293, accuracy = 0.98047
Epoch = 820, Batch = 429, loss = 0.00436, accuracy = 0.98047
Epoch = 821, Batch = 429, loss = 0.00919, accuracy = 0.98047
Epoch = 822, Batch = 429, loss = 0.00353, accuracy = 0.98047
Epoch = 823, Batch = 429, loss = 0.02173, accuracy = 0.98047
Epoch = 824, Batch = 429, loss = 0.00202, accuracy = 0.98047
Epoch = 825, Batch = 429

Epoch = 944, Batch = 429, loss = 0.00307, accuracy = 0.98047
Epoch = 945, Batch = 429, loss = 0.00234, accuracy = 0.98047
Epoch = 946, Batch = 429, loss = 0.00218, accuracy = 0.98047
Epoch = 947, Batch = 429, loss = 0.00695, accuracy = 0.98047
Epoch = 948, Batch = 429, loss = 0.00466, accuracy = 0.98047
Epoch = 949, Batch = 429, loss = 0.00352, accuracy = 0.98047
Epoch = 950, Batch = 429, loss = 0.00228, accuracy = 0.98047
Epoch = 951, Batch = 429, loss = 0.00648, accuracy = 0.98047
Epoch = 952, Batch = 429, loss = 0.00023, accuracy = 0.98047
Epoch = 953, Batch = 429, loss = 0.00176, accuracy = 0.98047
Epoch = 954, Batch = 429, loss = 0.00690, accuracy = 0.98047
Epoch = 955, Batch = 429, loss = 0.00044, accuracy = 0.98047
Epoch = 956, Batch = 429, loss = 0.00245, accuracy = 0.98047
Epoch = 957, Batch = 429, loss = 0.00311, accuracy = 0.98047
Epoch = 958, Batch = 429, loss = 0.01619, accuracy = 0.98047
Epoch = 959, Batch = 429, loss = 0.00387, accuracy = 0.98047
Epoch = 960, Batch = 429

In [ ]:
'''
import numpy as np

a = np.array([1, 2, 3, 4, 5, 6, np.inf])

np.any(np.isinf(a))
'''

In [3]:

# loading
import tensorflow as tf
# Remove the previous weights and bias
n_classes = 10
tf.reset_default_graph()

# Two Variables: weights and bias
# weights = tf.Variable(tf.truncated_normal([2, 3]), name = 'weight_1')
# bias = tf.Variable(tf.truncated_normal([3]), name = 'bias_1')
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'w1' : tf.Variable(tf.random_normal([7*7*64, 1024])),
    'w2' : tf.Variable(tf.random_normal([1024, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'b1' : tf.Variable(tf.random_normal([1024])),
    'b2' : tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Class used to save and/or restore Tensor Variables
save_file = './test1.ckpt'
saver = tf.train.Saver()

with tf.Session() as sess:
    # Load the weights and bias
    saver.restore(sess, save_file)
    
    # Show the values of weights and bias
    print('Weight:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(biases))
    

INFO:tensorflow:Restoring parameters from ./test1.ckpt
Weight:
{'wc1': array([[[[  5.30384719e-01,   7.70591676e-01,   9.98935997e-01,
            7.75745213e-01,   7.96322227e-01,  -2.85157442e-01,
           -7.64440894e-01,  -1.70114195e+00,  -1.31222212e+00,
            9.79790986e-01,   5.71447194e-01,  -8.49210382e-01,
            1.35728157e+00,  -1.29976913e-01,  -3.12664938e+00,
           -6.74573123e-01,   1.00536208e-05,  -4.65258718e-01,
            6.57077551e-01,   8.40132296e-01,  -5.93526602e-01,
           -3.64502132e-01,   7.89421380e-01,  -2.02460587e-01,
           -5.97088039e-01,  -5.92503667e-01,  -4.60111231e-01,
            5.95433235e-01,   6.71547711e-01,   6.28414035e-01,
           -1.43515718e+00,   3.62634867e-01]],

        [[ -8.53286982e-01,   2.16004562e+00,   3.26428145e-01,
           -1.50881851e+00,  -2.46281004e+00,  -1.80430329e+00,
           -5.44550180e-01,  -6.31653070e-01,   4.01538491e-01,
            7.34885931e-01,   1.43191791e+00,   